In [57]:
import csv
import os
import networkx as nx
from tqdm import tqdm
from operator import itemgetter

import roadmaptools.inout

In [3]:
# edges
edata = roadmaptools.inout.load_geojson(r"C:\AIC Experiment Data\VGA Manhattan\maps/edges.geojson")
edges = edata["features"]

[20:47:58]: Loading geojson file from: C:\AIC Experiment Data\VGA Manhattan\maps\edges.geojson


In [4]:
# real speeds
real_speed_dict = {}
with open(r"O:\AIC data\data\speeds/new_york-2018-Q2-19h.csv", 'r') as infile:
    reader = csv.reader(infile)
    
    # skip header
    header = next(reader)
    
    for row in tqdm(reader):
        real_speed_dict[(int(row[7]), int(row[8]))] = float(row[11])

112429it [00:00, 176776.51it/s]


In [19]:
# count missing values
missing = []
for edge in edges:
    properties = edge['properties']
    key = (int(properties['from_osm_id']), int(properties['to_osm_id']))
    if key not in real_speed_dict:
        missing.append(edge)
    else:
        properties['measured_speed'] = real_speed_dict[key]
        
len(missing)

2612

In [16]:
# create networkx graph
graph = nx.DiGraph()
for edge in edges:
    graph.add_edge(edge['properties']["from_osm_id"], edge['properties']["to_osm_id"], data=edge)

In [52]:
# count edges with speed records in neighborhood of each edge
missing_with_neighbor_speed_count = []
for edge in missing:
    neighbor_speed_count = 0
    for node in (edge['properties']['from_osm_id'], edge['properties']['to_osm_id']):
        for neighbor_edge in graph.edges(node, data=True):
            if 'measured_speed' in neighbor_edge[2]['data']['properties']:
                neighbor_speed_count +=1
    missing_with_neighbor_speed_count.append((edge, neighbor_speed_count))

In [60]:
# sort by neighborhood speed records
missing_with_neighbor_speed_count.sort(key=itemgetter(1), reverse=True)

In [66]:
# estimate speeds
for record in missing_with_neighbor_speed_count:
    neighbor_speed_count = 0
    neighbor_speed_sum = 0
    for node in (edge['properties']['from_osm_id'], edge['properties']['to_osm_id']):
        for neighbor_edge in graph.edges(node, data=True):
            if 'measured_speed' in neighbor_edge[2]['data']['properties']:
                neighbor_speed_count +=1
                neighbor_speed_sum += neighbor_edge[2]['data']['properties']['measured_speed']
    record[0]['properties']['measured_speed'] = neighbor_speed_sum / neighbor_speed_count

In [67]:
#check
for edge in edges:
    if 'measured_speed' not in edge['properties']:
        raise Exception

In [69]:
roadmaptools.inout.save_geojson(edata, r"C:\AIC Experiment Data\VGA Manhattan\maps/edges_with_measured_speed.geojson")

[22:31:16]: Saving geojson file to: C:\AIC Experiment Data\VGA Manhattan\maps/edges_with_measured_speed.geojson


In [65]:
missing_with_neighbor_speed_count[2250][1]

0

# Pseudo real speeds assignment

In [11]:
real_speed_sum = 0
count = 0
with open(r"O:\AIC data\data\speeds/new_york-2018-Q2-19h.csv", 'r') as infile:
    reader = csv.reader(infile)
    
    header = next(reader)
    
    for row in tqdm(reader):
        real_speed_sum += float(row[11])
        count += 1
        
avg_real_speed = real_speed_sum / count

112429it [00:00, 240226.21it/s]


In [5]:
posted_speed_sum = 0
for edge in edges:
    posted_speed_sum += int(edge["properties"]["maxspeed"])
avg_posted_speed = posted_speed_sum / len(edges)

In [12]:
avg_real_speed

18.66147654075017

In [13]:
avg_real_speed

18.66147654075017

In [14]:
avg_posted_speed

34.68234323432343

In [17]:
ratio = avg_posted_speed / avg_real_speed

In [18]:
ratio

1.8584994150162377

In [20]:
for edge in edges:
    edge["properties"]["maxspeed"] = int(round(int(edge["properties"]["maxspeed"]) / ratio))

In [21]:
roadmaptools.inout.save_geojson(edata, r"C:\AIC Experiment Data\VGA Manhattan\maps/edges.geojson")

[16:39:59]: Saving geojson file to: C:\AIC Experiment Data\VGA Manhattan\maps/edges.geojson
